In [6]:
import pytesseract
from PIL import Image
import os
from pdf2image import convert_from_path
import re
pdf_folder_path = './pdf/'
excel_template_path = 'template.xlsx'
output_excel_path = 'output.xlsx'
temp_img_path = "./temp_img/" 

""" 
    用于将pdf文件夹中的文件逐个提取成为图片 
    pdf_folder_path: pdf文件夹的路径
    return: images，提取出来的图像列表
"""
def pdf2img(pdf_folder_path):
    # 初始化列表用于存储从pdf文件中提取出来的图片
    images = []
    # for循环遍历pdf文件夹中的每个pdf文件并将其转换为原始图像存入列表
    for pdf_file in sorted(os.listdir(pdf_folder_path)):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, pdf_file)
            image_list_pdf = convert_from_path(pdf_path)
            image = image_list_pdf[0]
            images.append(image)     
    return images

""" 
    裁剪图片函数
    images: 图像列表
    return: img_cropped_list，裁剪后的图像列表
"""
def crop_images(images):
    # 初始化三个列表，用于存储裁剪后的图像（用户信息、测试数据以及数据曲线图）
    person_info_img_cropped_list = []
    test_data_img_cropped_list = []
    data_curve_img_cropped_list = []

    person_info_left = 245 # 用户信息左边界
    person_info_top = 360    # 用户信息上边界
    person_info_right = 1340  # 用户信息右边界
    person_info_bottom = 540 # 用户信息下边界

    test_data_left = 245 # 测试数据左边界
    test_data_top = 540    # 测试数据上边界
    test_data_right = 1050  # 测试数据右边界
    test_data_bottom = 1630 # 测试数据下边界

    data_curve_left = 1050 # 数据曲线图左边界
    data_curve_top = 540    # 数据曲线图上边界
    data_curve_right = 1340  # 数据曲线图右边界
    data_curve_bottom = 1630 # 数据曲线图下边界

    for i, image in enumerate(images):
        # 将图片保存以便进行OCR识别（也可以直接对PIL.Image对象进行处理）
        image.save(temp_img_path+f"page_{i+1}.png")

        # 打开原图片开始进行裁剪
        img = Image.open(temp_img_path+f"page_{i+1}.png")

        # 确保裁剪区域的坐标不会导致图像超出范围
        # 裁剪用户信息
        person_info_left = max(0, person_info_left)
        person_info_top = max(0, person_info_top)
        person_info_right = min(img.width, person_info_right)
        person_info_bottom = min(img.height, person_info_bottom)
        # 开始裁剪
        person_info_img_cropped = img.crop((person_info_left, person_info_top, person_info_right, person_info_bottom))
        person_info_img_cropped.save(temp_img_path+f"person_info_{i+1}_cropped.png")
        person_info_img_cropped_list.append(person_info_img_cropped)

        # 裁剪测试数据
        test_data_left = max(0, test_data_left)
        test_data_top = max(0, test_data_top)
        test_data_right = min(img.width, test_data_right)
        test_data_bottom = min(img.height, test_data_bottom)
        # 开始裁剪
        test_data_img_cropped = img.crop((test_data_left, test_data_top, test_data_right, test_data_bottom))
        test_data_img_cropped.save(temp_img_path+f"test_dat_{i+1}_cropped.png")
        test_data_img_cropped_list.append(test_data_img_cropped)

        # 裁剪数据曲线图
        data_curve_left = max(0, data_curve_left)
        data_curve_top = max(0, data_curve_top)
        data_curve_right = min(img.width, data_curve_right)
        data_curve_bottom = min(img.height, data_curve_bottom)
        # 开始裁剪
        data_curve_img_cropped = img.crop((data_curve_left, data_curve_top, data_curve_right, data_curve_bottom))
        data_curve_img_cropped.save(temp_img_path+f"data_curve_{i+1}_cropped.png")
        data_curve_img_cropped_list.append(data_curve_img_cropped)

        
    return person_info_img_cropped_list, test_data_img_cropped_list, data_curve_img_cropped_list
    
""" 
    使用Tesseract进行OCR识别 
    img_cropped_list: 裁剪后的图像列表
    return: person_info_list，识别的用户信息文本列表；test_data_list，识别的测试数据文本列表
""" 
def ocr(img_cropped_list):
    # 初始化一个文本列表用于存储每张图像提取出来的文本
    text_list = []
    # 用于存储两部分信息的列表
    person_info_list = []
    test_data_list = []

    for i, img_cropped in enumerate(img_cropped_list):
        text = pytesseract.image_to_string(img_cropped, lang='eng+chi_sim')
        text_list.append(text)
    
    # 分割用户信息与测试数据
    for i, text in enumerate(text_list):
        lines = text.splitlines()
        person_info = "\n".join(lines[:6])
        test_data = "\n".join(lines[6:])
        person_info_list.append(person_info)
        test_data_list.append(test_data)

    return person_info_list, test_data_list

"""  
    解析测试数据文本
    test_data_list: 识别的测试数据文本列表
"""
def parse_test_data(test_data_list):
    test_data_cropped_list = []
    for test_data in test_data_list:
        # 按行分割文本
        lines = test_data.splitlines()

        # 处理第二行和第三行
        date_info = lines[1].split(" ", 2)
        time_info = lines[2].split(" ", 2)

        # 初始化列表用于存储字典
        data_list = [date_info, time_info]

        # 编译正则表达式
        pattern = re.compile(r"(.+?) [\[\{\(](.+?)[\]\}\)] (.+)")

        # 处理剩下的每一行
        for line in lines[3:]:
            match = pattern.match(line)
            if match:
                data_name = match.group(1)
                unit = match.group(2)
                values = match.group(3).split()
                data_list.append({data_name: [unit] + values})
        test_data_cropped_list.append(data_list)

        for item in data_list:
            print(item)
        return test_data_cropped_list

def main():
    images = []
    images = pdf2img(pdf_folder_path)
    person_info_img_cropped_list, test_data_img_cropped_list, data_curve_img_cropped_list = crop_images(images)
    """ person_info_list, test_data_list = ocr(img_cropped_list)
    
    for person_info in person_info_list:
        print(person_info)
    print("--------------------------------------------------------")
    for test_data in test_data_list:
        print(test_data)
    parse_test_data(test_data_list) """

    


if __name__ == '__main__':
    main()